In [1]:
# skrypt do testowania biblioteki ffs

In [ ]:
from astropy.io import fits
from ffs import FFS

f = "~/Desktop/image.fits"

hdul = fits.open(f)
data = hdul[0].data
hdr = hdul[0].header
hdul.close()

ffs = FFS(data)
ffs.saturation = 50000
ffs.mk_stats()
ffs.sky_gradient()
ffs.find_stars(threshold=5, fwhm=3)
maska = data > np.median(data) + 3 * mad_std(data)
line_val, line_tht, line_r = ffs.hough_transform(maska)
fw,fwx,fwy,ell,th,cpe_t = ffs.star_info(box=15,N_stars=20)



In [ ]:
import os
import numpy as np


from astropy.time import Time
from astropy.stats import mad_std
import astropy.units as u
from astropy.coordinates import (SkyCoord, EarthLocation, AltAz, get_sun, get_body)

from astropy.io import fits

from ffs import FFS
ffs_make_segments = FFS.make_segments


path = "/work/vela/oca/fits/zb08/raw"
#path = "/work/vela/oca/fits/zb08/processed-ofp/science"
noce = [d for d in os.listdir(path) if (os.path.isdir(path+"/"+d) and not os.path.islink(path+"/"+d) )]

noce = sorted(noce)

print(noce)

In [2]:
fits_list = []
#nights = ["1040","1041","1042","1043","1044","1045","1046","1047","1048","1049"]
#nights = noce[-1]
nights = ["1057",]

for night in nights:
    for f in os.listdir(path+"/"+night):
        if os.path.isfile(path+"/"+night+"/"+f):
            fits_list.append(path+"/"+night+"/"+f)

fits_list = sorted(fits_list)
print(len(fits_list))

999


In [3]:
import ephem
from datetime import datetime

def moon_phase_pyephem(header):
    """
    Zwraca fazę Księżyca w % dla headera FITS.
    """
    obs = ephem.Observer()
    obs.lat = str(header["OBS-LAT"])
    obs.lon = str(header["OBS-LONG"])
    obs.elevation = header["OBS-ELEV"]

    # konwersja daty do formatu pyephem
    obs.date = datetime.fromisoformat(header["DATE-OBS"]).strftime("%Y/%m/%d %H:%M:%S")

    moon = ephem.Moon(obs)
    return moon.phase


In [22]:
import pickle
import warnings
from astropy.coordinates.baseframe import NonRotationTransformationWarning
warnings.filterwarnings("ignore",category=NonRotationTransformationWarning)


result = []

for f in fits_list:

    hdul = fits.open(f)
    data = hdul[0].data
    hdr = hdul[0].header
    hdul.close()

    if hdr["IMAGETYP"] == "science":
        location = EarthLocation(lat=hdr["OBS-LAT"] * u.deg, lon=hdr["OBS-LONG"] * u.deg, height=hdr["OBS-ELEV"] * u.m)
        obs_time = Time(hdr["DATE-OBS"], format="isot", scale="utc")
        field = SkyCoord(ra=hdr["RA_TEL"] * u.deg, dec=hdr["DEC_TEL"] * u.deg, frame="icrs")

        sun = get_sun(obs_time)
        moon = get_body("moon", obs_time, location)

        sun_sep = field.separation(sun)
        moon_sep = field.separation(moon)

        altaz_frame = AltAz(obstime=obs_time, location=location)
        sun_altaz = sun.transform_to(altaz_frame)
        moon_altaz = moon.transform_to(altaz_frame)


        moon_ph = moon_phase_pyephem(hdr)



        ffs = FFS(data)
        ffs.saturation = 50000
        ffs.mk_stats()
        ffs.sky_gradient()
        ffs.find_stars(threshold=5, fwhm=3)
        maska = data > np.median(data) + 3 * mad_std(data)
        line_val, line_tht, line_r = ffs.hough_transform(maska)
        fw,fwx,fwy,ell,th,cpe_t = ffs.star_info(box=15,N_stars=20)


        dqc = {}
        dqc["name"] = f
        dqc["hdr"] = hdr
        dqc["stats"] = ffs.stats
        dqc["sun_sep"] = sun_sep.deg
        dqc["moon_sep"] = moon_sep.deg
        dqc["sun_alt"] = sun_altaz.alt.deg
        dqc["moon_alt"] = moon_altaz.alt.deg
        dqc["moon_ph"] = moon_ph
        result.append(dqc)

        #print(f,len(result))

        #print(f,ffs.stats["stars"].keys())
        print(f, hdr["OBJECT"], np.nanmedian(ffs.stats["stars"]["fwhm"]))
        #print(diff_n_stars, diff_fwhm, diff_ell,diff_nc)


#with open("/home/mgorski/mglab/data/ffs_wyniki.pkl", "wb") as f:
#    pickle.dump(result, f)

# with open("/home/mgorski/mglab/data/ffs_wyniki_1050.pkl", "wb") as f:
#     pickle.dump(result, f)


/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55174.fits SA98 3.4704014037302535
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55212.fits SA98 2.9805595106496523
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55246.fits SA98 2.8821830828275714
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55281.fits SA98 3.393930205460907
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55315.fits SA98 3.0254305037507456
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55345.fits SA98 2.5451139917243504
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55380.fits SA98 2.594795994766636
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55460.fits RU149 3.0288080485091973
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55500.fits RU149 2.923693262666614
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55534.fits RU149 2.588844499001241
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55571.fits RU149 2.983910966698949
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55606.fits RU149 2.784586541669835
/work/vela/oca/fits/zb08/raw/1057/zb08c_1

KeyboardInterrupt: 

In [17]:
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55760.fits NGC2362 3.3071586002030395
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55774.fits NGC2362 3.299337755525796
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55790.fits NGC2362 3.4735555403631846
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55835.fits NGC2362 3.703094777317153
/work/vela/oca/fits/zb08/raw/1057/zb08c_1057_55881.fits NGC2362 3.982333123453013

846


In [23]:
with open("/home/mgorski/mglab/data/ffs_wyniki_1053.pkl", "wb") as f:
    pickle.dump(result, f)

In [1]:
print(ffs.stats_description)

NameError: name 'ffs' is not defined